# 深度神经网络#
**学习内容：**  
- 使用relu非线性激活  
- 搭建深层的神经网络(一层以上的隐层)  
- 实现一个易于使用的神经网络类  

## 需要调用的库 #
- dnn_utils 提供本次assignment的一些必要功能函数
- testCases 提供测试用例
- np.random.seed(1) 设置随机种子以便结果复现

In [3]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from depends.testCases_v3 import *
from depends.dnn_utils_v2 import sigmoid, sigmoid_backward, relu, relu_backward

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) 
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

np.random.seed(1)

## 流程 #
- 分别对一个2层NN与L层NN的参数进行初始化
- 实现前向传播模型：
    - 线性连接的前向传播
    - 激活函数
    - 集成前两个步骤
    - 循环调用模块完成L层NN的前向传播
- 计算损失
- 实现后向传播模型：
    - 线性连接的后向传播
    - 激活函数的导数
    - 集成前两个步骤
    - 循环调用模块完成L层NN的后向传播  
    
**流程图如下：**  
<img src = "source\Forward and Backward.png"></img>

## 参数初始化 #

### 2层神经网络的参数初始化 #
- 网络模型结构为：线性连接 -> relu激活 -> 线性连接 -> sigmoid函数
- 参数初始化用 `np.random.randn(shape)*0.01`
- 偏置初始化用 `np.zeros(shape)`

In [12]:
def initialize_parameters(n_x, n_h, n_y):
    
    np.random.seed(1)
    
    W1 = np.random.randn(n_h, n_x) * 0.01
    W2 = np.random.randn(n_y, n_h) * 0.01
    b1 = np.zeros([n_h, 1])
    b2 = np.zeros([n_y, 1])
    
    param = {"W1":W1,
             "W2":W2,
             "b1":b1,
             "b2":b2
    }
    
    return param

In [13]:
parameters = initialize_parameters(3,2,1)
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

W1 = [[ 0.01624345 -0.00611756 -0.00528172]
 [-0.01072969  0.00865408 -0.02301539]]
b1 = [[ 0.]
 [ 0.]]
W2 = [[ 0.01744812 -0.00761207]]
b2 = [[ 0.]]


## L层神经网络的初始化 #
L层网络的初始化需要知道每层的神经元数量，根据矩阵乘法：  
输入X的shape：(特征数, 样本量)  
W1的shape：(layer1的神经元数, X的特征数)  
W2的shape：(layer2的神经元数, layer1的神经元数)  
...  
WL的shape：(layerL-1的神经元数, layerL的神经元数)  
输出y^的shape：(layerL的神经元数, 1<font color = "blue">(如果是sigmoid函数)</font>)  

**这里函数接收一个list,记录每层隐层的神经元数量,其中 list[0] 与 list[-1] 代表 n_x 与 n_y(输入层不算一层，而输出层算一层) **

In [18]:
def initialize_parameters_deep(layer_dims):
    L = len(layer_dims)
    params = {}
    
    np.random.seed(3)
    for num in range(1, L):
        params["W" + str(num)] = np.random.randn(layer_dims[num], layer_dims[num - 1]) * 0.01
        params["b" + str(num)] = np.zeros([layer_dims[num], 1])
    
    return params

In [60]:
parameters = initialize_parameters_deep([5, 4, 3])
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

W1 = [[ 0.01788628  0.0043651   0.00096497 -0.01863493 -0.00277388]
 [-0.00354759 -0.00082741 -0.00627001 -0.00043818 -0.00477218]
 [-0.01313865  0.00884622  0.00881318  0.01709573  0.00050034]
 [-0.00404677 -0.0054536  -0.01546477  0.00982367 -0.01101068]]
b1 = [[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]
W2 = [[-0.01185047 -0.0020565   0.01486148  0.00236716]
 [-0.01023785 -0.00712993  0.00625245 -0.00160513]
 [-0.00768836 -0.00230031  0.00745056  0.01976111]]
b2 = [[ 0.]
 [ 0.]
 [ 0.]]


## 前向传播模块 #

### 线性链接的前向传播 #

对于第`L`层：  

$Z^{[l]} = W^{[l]}A^{[l-1]} +b^{[l]}$

其中, $A^{[0]} = X$

In [20]:
def linear_forward(A, W, b):
    Z = np.dot(W, A) + b
    cache = (A, W, b)
    return Z, cache

In [21]:
A, W, b = linear_forward_test_case()

Z, linear_cache = linear_forward(A, W, b)
print("Z = " + str(Z))

Z = [[ 3.26295337 -1.23429987]]


### 线性链接——激活函数的前向传播 #

线性链接的结果通过激活函数实现非线性，激活函数不改变shape。  

- **Sigmoid**: $\sigma(Z) = \sigma(W A + b) = \frac{1}{ 1 + e^{-(W A + b)}}$

- **ReLU**: The mathematical formula for ReLu is $A = RELU(Z) = max(0, Z)$  

这里提供了relu函数，它返回两个值，一个是激活后的值，一个是激活前的值(即Z，用于后向传播计算)

In [48]:
def linear_activation_forward(A_prev, W, b, activation):
    
    Z, linear_cache = linear_forward(A_prev, W, b)
    
    if activation == "sigmoid":
        A = 1 / (1 + np.exp(-Z)) 
        activation_cache = Z
        
    elif activation == "relu":
        A = relu(Z)[0]
        activation_cache = Z
        
    cache = (linear_cache, activation_cache)

    return A, cache

In [54]:
A_prev, W, b = linear_activation_forward_test_case()

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "sigmoid")
print("With sigmoid: A = " + str(A))

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "relu")
print("With ReLU: A = " + str(A))

With sigmoid: A = [[ 0.96890023  0.11013289]]
With ReLU: A = [[ 3.43896131  0.        ]]


## L层网络的前向传播#
**利用上面定义的linear_activation_forward**  
- 调用relu的linear_activation_forward (L-1) 次
- 调用sigmoid的linear_activation_forward 1 次  

流程图如下：  
<img src = "source\L layers NN Forward.png"></img>

parameters接收initialize_parameters_deep()返回的初始化参数，由于每层有`W`与`b`两个参数，因此层数为`len(paramters) // 2`

In [78]:
def L_model_forward(X, parameters):

    caches = []
    A = X
    L = len(parameters) // 2    
    
    for l in range(1, L):
        A_prev = A 
        W = parameters["W" + str(l)]
        b = parameters["b" + str(l)]
        A, cache = linear_activation_forward(A_prev, W, b, "relu")
        caches.append(cache)
    
    W = parameters["W" + str(L)]
    b = parameters["b" + str(L)]
    AL, cache = linear_activation_forward(A, W, b, "sigmoid")
    caches.append(cache)
            
    return AL, caches

In [79]:
X, parameters = L_model_forward_test_case_2hidden()
AL, caches = L_model_forward(X, parameters)
print("AL = " + str(AL))
print("Length of caches list = " + str(len(caches)))

AL = [[ 0.03921668  0.70498921  0.19734387  0.04728177]]
Length of caches list = 3


## 计算损失 #


利用$-\frac{1}{m} \sum\limits_{i = 1}^{m} (y^{(i)}\log\left(a^{[L] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right)$

In [106]:
def compute_cost(AL, Y):
    
    m = Y.shape[1]

    cost = -(1.0 / m) * np.sum(Y * np.log(AL) + (1 - Y) * np.log(1 - AL))
    
    cost = np.squeeze(cost)      
    
    return cost

In [107]:
Y, AL = compute_cost_test_case()

print("cost = " + str(compute_cost(AL, Y)))

cost = 0.414931599615


## 反向传播 #
**流程图如下：**  
<img src = "source\L layers NN Backward.png"></img>

### 线性链接的后向传播 #
由于$Z^{[l]} = W^{[l]} A^{[l-1]} + b^{[l]}$　　$dZ^{[l]} = \frac{\partial \mathcal{L} }{\partial Z^{[l]}}$  
求得：   
$ dW^{[l]} = \frac{\partial \mathcal{L} }{\partial W^{[l]}} = \frac{1}{m} dZ^{[l]} A^{[l-1] T}$  
$ db^{[l]} = \frac{\partial \mathcal{L} }{\partial b^{[l]}} = \frac{1}{m} \sum_{i = 1}^{m} dZ^{[l](i)}$  
$ dA^{[l-1]} = \frac{\partial \mathcal{L} }{\partial A^{[l-1]}} = W^{[l] T} dZ^{[l]}$  
`A.shape[1]`表示样本量，即m
<img src = "source\Linear backward.png"></img>  

In [110]:
def linear_backward(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]

    dW = np.dot(dZ, A_prev.T) / m
    db = np.sum(dZ, axis = 1, keepdims = True) / m
    dA_prev = np.dot(W.T, dZ)
    
    return dA_prev, dW, db

In [111]:
dZ, linear_cache = linear_backward_test_case()

dA_prev, dW, db = linear_backward(dZ, linear_cache)
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

dA_prev = [[ 0.51822968 -0.19517421]
 [-0.40506361  0.15255393]
 [ 2.37496825 -0.89445391]]
dW = [[-0.10076895  1.40685096  1.64992505]]
db = [[ 0.50629448]]


### 线性——激活的反向传播 #
由于 $A^{[l]} =  g(Z^{[l]})$  
可得 $dZ^{[l]} = dA^{[l]} * g'(Z^{[l]})$  
**这里提供了两个函数来计算激活函数的导数：**
- dZ = sigmoid_backward(dA, activation_cache)
- dZ = relu_backward(dA, activation_cache)

In [112]:
def linear_activation_backward(dA, cache, activation):
    
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)

    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    
    return dA_prev, dW, db

In [113]:
AL, linear_activation_cache = linear_activation_backward_test_case()

dA_prev, dW, db = linear_activation_backward(AL, linear_activation_cache, activation = "sigmoid")
print ("sigmoid:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db) + "\n")

dA_prev, dW, db = linear_activation_backward(AL, linear_activation_cache, activation = "relu")
print ("relu:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

sigmoid:
dA_prev = [[ 0.11017994  0.01105339]
 [ 0.09466817  0.00949723]
 [-0.05743092 -0.00576154]]
dW = [[ 0.10266786  0.09778551 -0.01968084]]
db = [[-0.05729622]]

relu:
dA_prev = [[ 0.44090989 -0.        ]
 [ 0.37883606 -0.        ]
 [-0.2298228   0.        ]]
dW = [[ 0.44513824  0.37371418 -0.10478989]]
db = [[-0.20837892]]


## L层网络的后向传播 #
<img src = "source\L layers NN Backward2.png"></img>
因为最后一层的导数要经过损失函数的导数，因此要先进行初始化：  
交叉熵的导数由 `dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))`求得  
由于最后一层的激活函数为sigmoid，也需要单独计算  

In [129]:
def L_model_backward(AL, Y, caches):

    grads = {}
    L = len(caches)
    m = AL.shape[1]
    #Y = Y.reshape(AL.shape)
    
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))

    current_cache = caches[L - 1]
    grads["dA" + str(L)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, "sigmoid")
    
    dA_prev_temp = grads["dA" + str(L)]
    for l in reversed(range(L-1)):

        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(dA_prev_temp, current_cache, "relu")
        grads["dA" + str(l + 1)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp

    return grads

In [130]:
AL, Y_assess, caches = L_model_backward_test_case()
grads = L_model_backward(AL, Y_assess, caches)
print_grads(grads)

dW1 = [[ 0.41010002  0.07807203  0.13798444  0.10502167]
 [ 0.          0.          0.          0.        ]
 [ 0.05283652  0.01005865  0.01777766  0.0135308 ]]
db1 = [[-0.22007063]
 [ 0.        ]
 [-0.02835349]]
dA1 = [[ 0.12913162 -0.44014127]
 [-0.14175655  0.48317296]
 [ 0.01663708 -0.05670698]]


## 参数更新#
$ W^{[l]} = W^{[l]} - \alpha \text{ } dW^{[l]}$  
$ b^{[l]} = b^{[l]} - \alpha \text{ } db^{[l]}$  
用一个for循环实现

In [136]:
def update_parameters(parameters, grads, learning_rate):

    L = len(parameters) // 2 
    
    for l in range(L):
        parameters["W" + str(l + 1)] -= learning_rate * grads["dW" + str(l + 1)]
        parameters["b" + str(l + 1)] -= learning_rate * grads["db" + str(l + 1)]
    
    return parameters

In [137]:
parameters, grads = update_parameters_test_case()
parameters = update_parameters(parameters, grads, 0.1)

print ("W1 = "+ str(parameters["W1"]))
print ("b1 = "+ str(parameters["b1"]))
print ("W2 = "+ str(parameters["W2"]))
print ("b2 = "+ str(parameters["b2"]))

W1 = [[-0.59562069 -0.09991781 -2.14584584  1.82662008]
 [-1.76569676 -0.80627147  0.51115557 -1.18258802]
 [-1.0535704  -0.86128581  0.68284052  2.20374577]]
b1 = [[-0.04659241]
 [-1.28888275]
 [ 0.53405496]]
W2 = [[-0.55569196  0.0354055   1.32964895]]
b2 = [[-0.84610769]]
